In [1]:
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from keras.regularizers import l1, l2
from tensorflow.keras import initializers
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import KFold
import keras_tuner
import keras
from keras import layers
import itertools
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping
from keras.callbacks import LearningRateScheduler
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
import keras.backend as K
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from math import ceil
import pickle

seed = 7
tf.random.set_seed(seed)

import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import KFold
from keras import layers
import itertools
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from math import ceil
import numpy as np
from pytorch_lightning.callbacks import LearningRateMonitor
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from pytorch_lightning.callbacks import ModelSummary
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from pytorch_lightning.loggers import CSVLogger
import pandas as pd
import datetime
import torch
import torch.nn as nn
from torchmetrics import Metric
import torch.utils.data as data
from torch.utils.data import Dataset, DataLoader 
import pytorch_lightning as pl
from datetime import datetime
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from torchmetrics import Accuracy
import logging
import os
from joblib import dump, load
logging.getLogger("lightning").addHandler(logging.NullHandler())
logging.getLogger("lightning").propagate = False
logging.getLogger("pytorch_lightning").setLevel(logging.WARNING)

2023-01-16 19:25:19.494621: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-16 19:25:21.722414: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-01-16 19:25:39.790755: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-16 19:25:39.805163: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [2]:
class LinearLearningRateScheduler(torch.optim.lr_scheduler._LRScheduler):
    def __init__(self, optimizer, lr_init, lr_final, epochs, last_epoch=-1):
        self.lr_init = lr_init
        self.lr_final = lr_final
        self.epochs = epochs
        super().__init__(optimizer, last_epoch=last_epoch)
    
    def step(self):
        self.last_epoch += 1
        epoch = self.last_epoch
        if epoch < self.epochs:
            alpha = epoch / self.epochs
            lr = self.lr_init * (1 - alpha) + alpha*self.lr_final
        else:
            lr = self.lr_final
        for param_group in self.optimizer.param_groups:
            param_group['lr'] = lr
            
    

def get_activation_function(name):
    if name == "relu":
        return nn.ReLU()
    elif name == "tanh":
        return nn.Tanh()
    elif name == "sigmoid":
        return nn.Sigmoid()
    elif name == "linear":
        return None

def mee(y_true, y_pred):
    return (y_true - y_pred).square().sum(axis=1).sqrt().mean()

def mse(y_true, y_pred):
    return (y_true - y_pred).square().sum(axis=1).mean()
    
class MEE(Metric):
    
    def __init__(self):
        super().__init__()
        self.add_state("result", default=torch.tensor(0.0))
        
    def update(self, y_pred, y_true):
        self.result += mee(y_true, y_pred)

    def compute(self):
        return self.result
    

class MultiLayerPerceptron(pl.LightningModule):
    def __init__(self, input_dim, output_dim, config):
        super(MultiLayerPerceptron, self).__init__()
        
        task = config["task"]
        self.lr = config["learning_rate"]
        self.momentum = config["momentum"]
        self.nesterov = config["nesterov"]
        init = config["initialization"]
        reg_method = config["regularization"]["method"]
        self.reg_method = reg_method
        self.lr_decay = config["learning_rate_decay"]
        
        
        
        if reg_method != None:
            value = config["regularization"]["value"]
            self.reg_value = value
        
        self.task = task
        self.model = nn.Sequential()
        
        units = config["units_1"]
        
        self.model.add_module("linear_input", nn.Linear(input_dim, units))
        
        n_layers = 0
        for key in config.keys():
            if key.startswith("units_"):
                n_layers += 1
                
        for n_layer in range(1, n_layers+1):
            out_units = config[f"units_{n_layer}"]
            self.model.add_module(f"linear_{n_layer}", nn.Linear(units, out_units))
            activation = get_activation_function(config[f"activations_{n_layer}"])
            if activation != None:
                self.model.add_module(f"activations_{n_layer}", activation)
            units = out_units
        
        if task == "classification":
            self.model.add_module("layer_output", nn.Linear(units, output_dim))
            self.model.add_module("activations_output", nn.Sigmoid())
            self.metric = Accuracy()
            self.metric_name = "accuracy"
        
        if task == "regression":
            self.model.add_module("layer_output", nn.Linear(units, output_dim))
            self.metric = MEE()
            self.metric_name = "mee"
                    
        if init["type"] == "uniform":
            self.min = init["min"]
            self.max = init["max"]
            self.apply(self.init_weights_uniform)
        elif init["type"] == "normal":
            self.mean = init["mean"]
            self.std = init["std"]
            self.apply(self.init_weights_normal)
            
            
        self.loss = nn.MSELoss()
        self.dicts = []
            
            

    def init_weights_uniform(self, m):
        if isinstance(m, nn.Linear):
            torch.nn.init.uniform_(m.weight, self.min, self.max)
            torch.nn.init.uniform_(m.bias, self.min, self.max)
            # m.bias.data.fill_(0.01)
            

    def init_weights_normal(self, m):
        if isinstance(m, nn.Linear):
            torch.nn.init.normal_(m.weight, self.mean, self.std)
            torch.nn.init.normal_(m.bias, self.mean, self.std)
            # m.bias.data.fill_(0.01)
            
            
    def compute_metric(self, y_pred, y_true):
        if self.task == "classification":
            y_pred = y_pred.reshape(y_true.shape)  
            return self.metric(y_pred, y_true.to(torch.int32))
        elif self.task == "regression":
            y_true = y_true.to(torch.float32)
            return self.metric(y_pred, y_true)


    def compute_loss(self, y_pred, y_true):
        if self.task == "classification":
            y_pred = y_pred.reshape(y_true.shape)  
            return self.loss(y_pred, y_true)
        elif self.task == "regression":
            y_true = y_true.to(torch.float32)
            return self.loss(y_pred, y_true)
        
        
    def get_regularization_term(self):
        if self.reg_method != None:
            
            value = self.reg_value
            p = 1 if self.reg_method == "l1" else 2
            reg = 0
            
            for param in self.model.parameters():
                reg += torch.linalg.norm(param, p)
                
            reg *= value
            
            return reg
        
        return 0
            
        
        
    def forward(self, x):
        return self.model(x)

    
    def configure_optimizers(self):
        optimizer = torch.optim.SGD(self.parameters(), lr=self.lr, momentum=self.momentum, nesterov=self.nesterov)
        if self.lr_decay != None:
            decay_epochs = self.lr_decay["epochs"]
            lr_final = self.lr_decay["lr_final"]
            lr_init = self.lr
            scheduler = LinearLearningRateScheduler(optimizer, lr_init, lr_final, decay_epochs)
            return [optimizer], [scheduler]
            
        return optimizer
    
    
    

    def training_step(self, train_batch, batch_idx):
        X, y = train_batch 
        y_copy = y # Integer y for the accuracy
        X = X.type(torch.float32)
        y = y.type(torch.float32)  
        # forward pass
        y_pred = self.forward(X)
        # accuracy
        acc = self.compute_metric(y_pred, y_copy)
        # compute loss
        reg = self.get_regularization_term()
        loss = self.compute_loss(y_pred, y) + reg
        self.log_dict({'loss': loss, f'{self.metric_name}': acc}, on_step=False, on_epoch=True, prog_bar=True, logger=True)
        return loss
    
    
    
    def validation_step(self, validation_batch, batch_idx):
        X, y = validation_batch
        X = X.type(torch.float32)
        # forward pass
        y_pred = self.forward(X)
        # compute metrics 
        acc = self.compute_metric(y_pred, y)
        # compute loss
        loss = self.compute_loss(y_pred, y)
        self.log_dict({'val_loss': loss, f'val_{self.metric_name}': acc}, prog_bar=True, logger=True)
        return loss
    
    
    
    
    def test_step(self, test_batch, batch_idx):
        X, y = test_batch
        X = X.type(torch.float32)
        # forward pass
        y_pred = self.forward(X)
        # compute metrics 
        acc = self.compute_metric(y_pred, y)
        # compute loss
        loss = self.compute_loss(y_pred, y)
        self.log_dict({'test_loss': loss, f'test_{self.metric_name}': acc}, on_epoch=True,on_step=False, prog_bar=True, logger=True)
        return loss

In [3]:
x_scaler = None
y_scaler = None

def load_dataset(scale=True):
    global x_scaler
    global y_scaler
    
    x_scaler = StandardScaler()
    y_scaler = StandardScaler()
    
    train = pd.read_csv("../data/ML-CUP22-INTERNAL-TR.csv", header=None, sep=",")
    test = pd.read_csv("../data/ML-CUP22-INTERNAL-TS.csv", header=None, sep=",")
    train.drop(0, axis=1, inplace=True)
    test.drop(0, axis=1, inplace=True)

    x_train = train.iloc[:, :9].to_numpy().astype(np.float64)
    y_train = train.iloc[:, 9:].to_numpy().astype(np.float64)
    x_test = test.iloc[:, :9].to_numpy().astype(np.float64)
    y_test = test.iloc[:, 9:].to_numpy().astype(np.float64)
    
    blind_test = pd.read_csv("../data/ML-CUP22-TS.csv", header=None, sep=",")
    blind_test.drop(0, axis=1, inplace=True)

    x_blind_test = blind_test.iloc[:, :9].to_numpy().astype(np.float64)    
    
    if scale:
        x_scaler.fit(x_train)
        x_train = x_scaler.transform(x_train)
        x_test = x_scaler.transform(x_test)
        x_blind_test = x_scaler.transform(x_blind_test)
        
        y_scaler.fit(y_train)
        y_train = y_scaler.transform(y_train)
        y_test = y_scaler.transform(y_test)     
        
    return x_train, y_train, x_test, y_test, x_blind_test

In [4]:
def get_mee(y_true, y_pred):
    return np.mean(np.sqrt(np.sum(np.square(y_true - y_pred), axis=1)))

def get_neural_networks():
    keras_model = keras.models.load_model("./../models/keras_model.hdf5", custom_objects={"mee": mee})
    torch_model = torch.load("./../models/pytorch_model.pt")
    random_model = keras.models.load_model('./../models/randomized_model/', custom_objects={"mee": mee})
    return keras_model, torch_model, random_model

def get_trees():
    dt_model = load('./../models/dt_mlcup.z')
    rf_model = load('./../models/rf_mlcup.z')
    return dt_model, rf_model
    


def get_keras_pred(model, x):
    global y_scaler
    
    y = model.predict(x)
    y = y_scaler.inverse_transform(y)
    return y

def get_torch_pred(model, x):
    y = model(torch.from_numpy(x).to(torch.float32))
    y = y.detach().numpy()
    y = y_scaler.inverse_transform(y)
    return y

In [5]:
x_train, y_train, x_test, y_test, x_blind_test = load_dataset(False)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)
print(x_blind_test.shape)

(1194, 9) (1194, 2)
(298, 9) (298, 2)
(529, 9)


In [6]:
x_train_scaled, y_train_scaled, x_test_scaled, y_test_scaled, x_blind_test_scaled = load_dataset(True)
print(x_train_scaled.shape, y_train_scaled.shape)
print(x_test_scaled.shape, y_test_scaled.shape)
print(x_blind_test_scaled.shape)

(1194, 9) (1194, 2)
(298, 9) (298, 2)
(529, 9)


In [10]:
x_train

array([[-1.704815,  1.503106, -0.817083, ...,  0.217795,  0.462728,
        -1.237246],
       [-0.888962,  0.711473, -0.480023, ..., -0.445018,  1.719143,
        -1.09625 ],
       [ 1.143802, -1.591273,  0.670477, ..., -0.694223, -1.321391,
         0.662656],
       ...,
       [ 1.589704, -1.90002 ,  0.415666, ..., -1.226479, -1.350128,
        -0.010998],
       [ 0.22256 ,  0.171394,  2.338296, ...,  0.61909 ,  0.448008,
         0.134578],
       [-1.701126,  0.88336 , -0.332723, ...,  0.032551,  0.518337,
        -1.39578 ]])

In [7]:
keras_model, torch_model, random_model = get_neural_networks()
dt_model, rf_model = get_trees()



keras_pred = get_keras_pred(keras_model, x_test_scaled)
rand_pred = get_keras_pred(random_model, x_test_scaled)
torch_pred = get_torch_pred(torch_model, x_test_scaled)
dt_pred = dt_model.predict(x_test)
rf_pred = dt_model.predict(x_test)

preds = [keras_pred, rand_pred, torch_pred, dt_pred, rf_pred]

mean_pred = sum(preds)/len(preds)

2023-01-16 19:26:00.747593: E tensorflow/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2023-01-16 19:26:00.747697: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: diego
2023-01-16 19:26:00.747736: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: diego
2023-01-16 19:26:00.748141: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 515.86.1
2023-01-16 19:26:00.748235: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 515.86.1
2023-01-16 19:26:00.748259: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 515.86.1
2023-01-16 19:26:00.749680: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operatio

10/10 [==============================] - 0s 1ms/step


In [8]:
rf_pred

array([[-1.42873867, 23.58747375],
       [ 1.21445267,  8.32434575],
       [ 1.26945238,  6.42741423],
       [ 0.952138  ,  8.98544693],
       [-0.37447992, 16.45796792],
       [ 1.21445267,  8.32434575],
       [-0.76815892, 15.74043215],
       [ 0.67414658, 10.86678542],
       [ 1.26945238,  6.42741423],
       [-1.01563884, 18.50069111],
       [-0.78549767, 15.424459  ],
       [-1.1407675 , 20.7471125 ],
       [-1.01563884, 18.50069111],
       [-1.1407675 , 20.7471125 ],
       [ 0.90082909,  4.52204453],
       [ 0.51979573, 10.1608616 ],
       [ 1.26945238,  6.42741423],
       [-1.46602071, 22.5010565 ],
       [-1.01563884, 18.50069111],
       [-1.01563884, 18.50069111],
       [-1.1407675 , 20.7471125 ],
       [ 1.26945238,  6.42741423],
       [ 0.67414658, 10.86678542],
       [-1.1407675 , 20.7471125 ],
       [-1.17344776, 21.89466153],
       [ 1.21445267,  8.32434575],
       [ 0.67414658, 10.86678542],
       [ 1.26945238,  6.42741423],
       [ 0.63475263,

In [92]:
rand_pred

array([[  7.8281884, -28.098131 ],
       [ 20.837502 , -27.711039 ],
       [ 18.308386 , -27.584085 ],
       [ 14.837602 , -29.840246 ],
       [  4.2629166, -24.533525 ],
       [ 18.899696 , -28.656515 ],
       [  7.397344 , -27.774761 ],
       [ 12.123469 , -30.789919 ],
       [ 22.618519 , -26.904491 ],
       [  4.256721 , -22.737694 ],
       [  9.293579 , -29.552355 ],
       [  4.5974207, -18.06229  ],
       [  6.2362742, -25.782257 ],
       [  4.363928 , -19.792564 ],
       [ 18.935413 , -27.90587  ],
       [  9.714448 , -29.282492 ],
       [ 21.440811 , -27.576033 ],
       [  4.475519 , -23.813484 ],
       [  5.556927 , -25.505188 ],
       [  4.508066 , -23.216352 ],
       [  4.433313 , -18.437494 ],
       [ 19.206709 , -28.201818 ],
       [ 13.96017  , -30.381685 ],
       [  4.4266496, -18.473589 ],
       [  5.7466946, -26.930685 ],
       [ 16.679697 , -29.322239 ],
       [ 13.287283 , -29.135664 ],
       [ 16.999054 , -28.952871 ],
       [  9.282754 ,